In [9]:
import sys, os
sys.path.insert(0,'../')

In [10]:
import jsonlines
import numpy as np

from src.questions_construction.domains import ALL_DOMAIN_CLASSES

def open_jsonl(path):
    with jsonlines.open(path, 'r') as r:
        data = [obj for obj in r]
    return data

In [11]:
domain = 'blocksworld'
data = open_jsonl(f'/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/states_actions/{domain}/Instance_1.jsonl')

64

{'part_of_plan?': False,
 'fluents': [],
 'neg_fluents': [],
 'executable?': False}

In [20]:
# domain = ALL_DOMAIN_CLASSES[2].DOMAIN_NAME
from collections import defaultdict

def num_fluents(data):
    return len(data[0]['action_init']['fluents']) + len(data[0]['action_init']['neg_fluents'])


fluent_count_by_domain = defaultdict(list)
stats_by_domain = {}
for domain in sorted([c.DOMAIN_NAME for c in ALL_DOMAIN_CLASSES]):
    action_lengths = []
    objects = []
    for i in range(1, 11):
        data = open_jsonl(f'/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/states_actions/{domain}/Instance_{i}.jsonl')
        fluent_count_by_domain[domain].append(num_fluents(data))
    mean, std = np.mean(fluent_count_by_domain[domain]), np.std(fluent_count_by_domain[domain])
    stats_by_domain[domain] = {'mean': mean, 'std': std}
    
    print(domain)
    print(stats_by_domain[domain])
    # print(objects)
    # print(f'{round(mean, 2)} ± {round(std, 2)}')
    print('\n')

blocksworld
{'mean': 83.5, 'std': 14.994999166388773}

depots
{'mean': 218.9, 'std': 25.327652871910576}

driverlog
{'mean': 313.9, 'std': 95.79086595286631}

goldminer
{'mean': 348.3, 'std': 174.43053058452813}

grippers
{'mean': 68.9, 'std': 21.897260102579047}

logistics
{'mean': 96.0, 'std': 25.64371267971937}

miconic
{'mean': 238.4, 'std': 43.2601433192263}

mystery
{'mean': 122.2, 'std': 18.197802065084673}

npuzzle
{'mean': 153.0, 'std': 0.0}

satellite
{'mean': 197.2, 'std': 54.19188131076462}

spanner
{'mean': 260.9, 'std': 15.12249979335427}

visitall
{'mean': 604.2, 'std': 154.20622555526091}

zenotravel
{'mean': 95.4, 'std': 7.418894796396563}


In [29]:
sorted(stats_by_domain.items(), key=lambda x: x[1]['mean'])

[('grippers', {'mean': 68.9, 'std': 21.897260102579047}),
 ('blocksworld', {'mean': 83.5, 'std': 14.994999166388773}),
 ('zenotravel', {'mean': 95.4, 'std': 7.418894796396563}),
 ('logistics', {'mean': 96.0, 'std': 25.64371267971937}),
 ('mystery', {'mean': 122.2, 'std': 18.197802065084673}),
 ('npuzzle', {'mean': 153.0, 'std': 0.0}),
 ('satellite', {'mean': 197.2, 'std': 54.19188131076462}),
 ('depots', {'mean': 218.9, 'std': 25.327652871910576}),
 ('miconic', {'mean': 238.4, 'std': 43.2601433192263}),
 ('spanner', {'mean': 260.9, 'std': 15.12249979335427}),
 ('driverlog', {'mean': 313.9, 'std': 95.79086595286631}),
 ('goldminer', {'mean': 348.3, 'std': 174.43053058452813}),
 ('visitall', {'mean': 604.2, 'std': 154.20622555526091})]

In [28]:
list(stats_by_domain.items())[0][1]['mean']

83.5